In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_3c4e2af75bb0495ab77132958276fffa.csv"
df=pd.read_csv(url)
df.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,35400,CTG0001.txt,240,357,120,120,0,0,0,...,0,0,0,0,0,0,1,0,9,2
1,Fmcs_1.txt,35188,CTG0002.txt,5,632,132,132,4,0,4,...,0,0,0,1,0,0,0,0,6,1
2,Fmcs_1.txt,35188,CTG0003.txt,177,779,133,133,2,0,5,...,0,0,0,1,0,0,0,0,6,1
3,Fmcs_1.txt,35188,CTG0004.txt,411,1192,134,134,2,0,6,...,0,0,0,1,0,0,0,0,6,1
4,Fmcs_1.txt,35188,CTG0005.txt,533,1147,132,132,4,0,5,...,0,0,0,0,0,0,0,0,2,1


In [2]:
df=df[["LB","AC","FM","UC","DL","DS","DP","ASTV","MSTV","ALTV","MLTV","Width","Min","Max","Nmax","Nzeros","Mode","Mean","Median","Variance","Tendency","NSP"]]
df.head()

,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
0,120,0,0,0,0,0,0,73,0.5,43,...,62,126,2,0,120,137,121,73,1,2
1,132,4,0,4,2,0,0,17,2.1,0,...,68,198,6,1,141,136,140,12,0,1
2,133,2,0,5,2,0,0,16,2.1,0,...,68,198,5,1,141,135,138,13,0,1
3,134,2,0,6,2,0,0,16,2.4,0,...,53,170,11,0,137,134,137,13,1,1
4,132,4,0,5,0,0,0,16,2.4,0,...,53,170,9,0,137,136,138,11,1,1


In [3]:
X=df.drop(["NSP"],axis=1)
y=df[["NSP"]]

In [4]:
y.groupby("NSP")["NSP"].count()

NSP
1    1655
2     295
3     176
Name: NSP, dtype: int64

In [5]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
6,539.496929
9,345.156385
7,343.820419
17,297.625497
16,275.117696
18,248.772237
1,167.889598
19,150.796849
0,140.621076
8,119.882006


In [6]:
X_scores=scores[scores["scores"]>=100]
len(X_scores)

10

In [7]:
X_prime=X.iloc[:,X_scores.index]
X_prime.head()

,DP,ALTV,ASTV,Mean,Mode,Median,AC,Variance,LB,MSTV
0,0,43,73,137,120,121,0,73,120,0.5
1,0,0,17,136,141,140,4,12,132,2.1
2,0,0,16,135,141,138,2,13,133,2.1
3,0,0,16,134,137,137,2,13,134,2.4
4,0,0,16,136,137,138,4,11,132,2.4


In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_prime=pd.DataFrame(scaler.fit_transform(X_prime),columns=X_prime.columns)
X_prime.head()

,DP,ALTV,ASTV,Mean,Mode,Median,AC,Variance,LB,MSTV
0,-0.27153,1.802542,1.513190,0.153270,-1.065614,-1.181642,-0.764740,1.870569,-1.352220,-0.943095
1,-0.27153,-0.535361,-1.744751,0.089126,0.216638,0.132038,0.358852,-0.234998,-0.132526,0.868841
2,-0.27153,-0.535361,-1.802928,0.024982,0.216638,-0.006244,-0.202944,-0.200481,-0.030884,0.868841
3,-0.27153,-0.535361,-1.802928,-0.039162,-0.027600,-0.075385,-0.202944,-0.200481,0.070757,1.208579
4,-0.27153,-0.535361,-1.802928,0.089126,-0.027600,-0.006244,0.358852,-0.269516,-0.132526,1.208579


In [54]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1594, 10)
(532, 10)


In [16]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BalancedRandomForestClassifier(criterion="entropy",bootstrap=False,sampling_strategy="not majority",n_jobs=-1,
                                       replacement=False,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(bootstrap=False, criterion='entropy', n_jobs=-1,
                               random_state=0,
                               sampling_strategy='not majority')

In [17]:
from sklearn.metrics import accuracy_score
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.9661654135338346


In [18]:
from sklearn.metrics import f1_score
print("F1 score on train data: ",f1_score(y_train,train_pred,average="micro"))
print("F1 score on test data: ",f1_score(y_test,test_pred,average="micro"))

F1 score on train data:  1.0
F1 score on test data:  0.9661654135338346


In [19]:
from sklearn.metrics import precision_score
print("Precision score on train data: ",precision_score(y_train,train_pred,average="micro"))
print("Precision score on test data: ",precision_score(y_test,test_pred,average="micro"))

Precision score on train data:  1.0
Precision score on test data:  0.9661654135338346


In [20]:
from sklearn.metrics import recall_score
print("Recall score on train data: ",recall_score(y_train,train_pred,average="micro"))
print("Recall score on test data: ",recall_score(y_test,test_pred,average="micro"))

Recall score on train data:  1.0
Recall score on test data:  0.9661654135338346


In [21]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[1242    0    0]
 [   0  213    0]
 [   0    0  139]]
Confusion matrix for test data: 
 [[409   3   1]
 [ 11  69   2]
 [  1   0  36]]


In [107]:
X_scores=scores[scores["scores"]>=150]
len(X_scores)

8

In [108]:
X_prime=X.iloc[:,X_scores.index]
X_prime.head()

,DP,ALTV,ASTV,Mean,Mode,Median,AC,Variance
0,0,43,73,137,120,121,0,73
1,0,0,17,136,141,140,4,12
2,0,0,16,135,141,138,2,13
3,0,0,16,134,137,137,2,13
4,0,0,16,136,137,138,4,11


In [109]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_prime=pd.DataFrame(scaler.fit_transform(X_prime),columns=X_prime.columns)
X_prime.head()

,DP,ALTV,ASTV,Mean,Mode,Median,AC,Variance
0,-0.27153,1.802542,1.513190,0.153270,-1.065614,-1.181642,-0.764740,1.870569
1,-0.27153,-0.535361,-1.744751,0.089126,0.216638,0.132038,0.358852,-0.234998
2,-0.27153,-0.535361,-1.802928,0.024982,0.216638,-0.006244,-0.202944,-0.200481
3,-0.27153,-0.535361,-1.802928,-0.039162,-0.027600,-0.075385,-0.202944,-0.200481
4,-0.27153,-0.535361,-1.802928,0.089126,-0.027600,-0.006244,0.358852,-0.269516


In [110]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1594, 8)
(532, 8)


In [111]:
train=pd.concat([X_train,y_train],axis=1)
train1=train[train["NSP"]==1]
train2=train[train["NSP"]==2]
train2=pd.concat([train2]*2,axis=0)
train3=train[train["NSP"]==3]
train=pd.concat([train1,train2,train3],axis=0)
X_train=train.drop(["NSP"],axis=1)
y_train=train[["NSP"]]

In [112]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(criterion="entropy",max_features="log2",bootstrap=False,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(bootstrap=False, criterion='entropy',
                       max_features='log2', random_state=0)

In [113]:
from sklearn.metrics import accuracy_score
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  1.0
Accuracy on test data:  0.9624060150375939


In [114]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for train data: \n",confusion_matrix(y_train,train_pred))
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for train data: 
 [[1242    0    0]
 [   0  426    0]
 [   0    0  139]]
Confusion matrix for test data: 
 [[407   4   2]
 [ 10  71   1]
 [  2   1  34]]
